#### Importação das bibliotecas necessárias e adição de um novo diretório ao caminho de busca dos módulos

In [1]:
import sys
sys.path.append('/home/jovyan/work')
from utils.SparkRefine import SparkRefine
from utils.utils import Utils

#### Cria uma instância da classe SparkRefine e outra da classe Utils

In [2]:
spark_refine = SparkRefine()
utils = Utils()

#### Define uma variável e cria uma visualização temporária para consultas SQL aplicando algumas transformações usando a função 'create_spark_temp_view_with_transform' cujos parâmetros especificam o nome da vizualização, o caminho do arquivo Parquet e as transformações realizadas

In [ ]:
bucket_name = 'refined'
business_area = 'sei_andamentos'
source_system = 'operacional'

table_dim_sei_andamentos = "dim_sei_andamentos"

spark_refine.create_spark_temp_view(
    bucket_name=bucket_name,
    file_path=f'{business_area}/{source_system}/{table_dim_sei_andamentos}.parquet', 
    view_name=table_dim_sei_andamentos)

#### Define uma variável e cria uma visualização temporária para consultas SQL usando a função 'create_spark_temp_view' cujos parâmetros especificam o nome da vizualização e o caminho do arquivo Parquet

In [ ]:
bucket_name = 'refined'
business_area = 'sei_unidades'
source_system = 'operacional'

table_dim_sei_unidades = "dim_sei_unidades"

spark_refine.create_spark_temp_view(
    bucket_name=bucket_name,
    file_path=f'{business_area}/{source_system}/{table_dim_sei_unidades}.parquet', 
    view_name=table_dim_sei_unidades)

#### Define uma variável e cria uma visualização temporária para consultas SQL usando a função 'create_spark_temp_view' cujos parâmetros especificam o nome da vizualização e o caminho do arquivo Parquet

In [ ]:
bucket_name = 'refined'
business_area = 'sei_procedimentos'
source_system = 'operacional'

table_dim_sei_procedimentos = "dim_sei_procedimentos"

spark_refine.create_spark_temp_view(
    bucket_name=bucket_name,
    file_path=f'{business_area}/{source_system}/{table_dim_sei_procedimentos}.parquet', 
    view_name=table_dim_sei_procedimentos)

#### Define uma variável e cria uma visualização temporária para consultas SQL usando a função 'create_spark_temp_view' cujos parâmetros especificam o nome da vizualização e o caminho do arquivo Parquet

In [ ]:
bucket_name = 'refined'
business_area = 'sei_tipo_procedimentos'
source_system = 'operacional'

table_dim_sei_tipo_procedimentos = "dim_sei_tipo_procedimentos"

spark_refine.create_spark_temp_view(
    bucket_name=bucket_name,
    file_path=f'{business_area}/{source_system}/{table_dim_sei_tipo_procedimentos}.parquet', 
    view_name=table_dim_sei_tipo_procedimentos)

#### Utiliza a função 'generate_parquet_dataframe_from_sql' para transformar o resultado da consulta SQL em um DataFrame PySpark e, em seguida, salvar esse DataFrame em formato Parquet

In [ ]:
spark_refine.generate_parquet_dataframe_from_sql(
    sql=f"""
        SELECT 
            a.IdProcedimentoFormatado,
            u.Sigla,
            tp.Nome AS tipo,
            DATEDIFF(CURRENT_TIMESTAMP(), TO_DATE(max_dates.max_data, 'dd/MM/yyyy HH:mm:ss')) AS Dias_Sem_Movimentacao
        FROM {table_dim_sei_andamentos} a
            LEFT JOIN {table_dim_sei_unidades} u ON u.IdUnidade = a.Unidade
            LEFT JOIN {table_dim_sei_procedimentos} p ON p.ProcedimentoFormatado = a.IdProcedimentoFormatado
            LEFT JOIN {table_dim_sei_tipo_procedimentos} tp ON tp.IdTipoProcedimento = p.TipoProcedimento
        INNER JOIN (
            SELECT 
                Unidade, 
                IdProcedimentoFormatado,
                MAX(TO_TIMESTAMP(DataHora, 'dd/MM/yyyy HH:mm:ss')) AS max_data
            FROM {table_dim_sei_andamentos}
            WHERE 
                IdProcedimentoFormatado IN (
                    SELECT ProcedimentoFormatado FROM {table_dim_sei_procedimentos}
                )
                AND IdTarefa IN ('2')
            GROUP BY 
                Unidade, IdProcedimentoFormatado
        ) max_dates 
        ON a.Unidade = max_dates.Unidade 
        AND a.IdProcedimentoFormatado = max_dates.IdProcedimentoFormatado
        AND TO_TIMESTAMP(a.DataHora, 'dd/MM/yyyy HH:mm:ss') = max_dates.max_data
        WHERE 
            a.IdProcedimentoFormatado IN (
                SELECT ProcedimentoFormatado FROM {table_dim_sei_procedimentos}
            )
            AND a.IdTarefa IN ('2')
            AND u.Sigla != 'AL-SAI'
            AND tp.Nome IN (
                'Adesão à Ata de Registro de Preços', 'Compra de Material e Contratação de Serviços',
                'Compra Direta', 'Contratação por Pregão Eletrônico',
                'Contratação por Pregão Presencial', 'Contratação/Pagamento de Serviços',
                'Contrato', 'Controle de Pagamento de Registro de Preço',
                'Convênios', 'Dispensa de Licitação',
                'Dispensa de Licitação - Extrato', 'Inexigibilidade de Licitação',
                'Licitação', 'Pagamento de Despesas Continuadas/Contratos',
                'Prestação de Serviços', 'Projeto/Proposta',
                'Projetos/Obras e Construções', 'Registro de Preço'
            )
        ORDER BY max_dates.max_data DESC
    """
)


#### Define a variável com o novo nome da tabela e utiliza a função 'send_dataframe_to_minio' para enviar a tabela para armazenamento no MinIO

In [ ]:
bucket_name = 'refined'
business_area = 'sei_processos_parados'
source_system = 'operacional'

In [ ]:
table_fato_sei_processos_parados = "fato_sei_processos_parados"

spark_refine.send_dataframe_to_minio(
    f'{business_area}/{source_system}/{table_fato_sei_processos_parados}.parquet', 
)